# Multiword expressions identification and extraction

In [1]:
import collections
import math
import os
import requests
from spacy.lang.pl import Polish
import time

1. Use SpaCy tokenizer API to tokenize the text from the law corpus.

In [4]:
nlp = Polish()
tokenizer = nlp.tokenizer

In [5]:
text_tokenized = {}

for root, _, files in os.walk('./ustawy'):
    for file_name in files:
        path = os.path.join(root, file_name)
        with open(path, encoding='utf-8') as file:
            content = file.read()
            tokens = [
                token.text.lower().strip()
                for token
                in tokenizer(content)
            ]
            text_tokenized[path] = [token for token in tokens if token != '']

2. Compute bigram counts of downcased tokens.

Given the sentence: "The quick brown fox jumps over the
   lazy dog.", the bigram counts are as follows:
   1. "the quick": 1
   1. "quick brown": 1
   1. "brown fox": 1
   1. ...
   1. "dog .": 1

In [6]:
def count_ngrams(tokens, n=2):
    ngram_dict = collections.defaultdict(int)
    for words in zip(*[tokens[offset:] for offset in (range(n))]):
        ngram_dict[' '.join(words)] += 1
    return ngram_dict

In [7]:
bigram_counts = collections.defaultdict(int)

for file, tokens in text_tokenized.items():
    for bigram, count in count_ngrams(tokens).items():
        bigram_counts[bigram] += count

In [8]:
sorted(bigram_counts.items(), key=lambda bigram_count: -bigram_count[1])[:20]


[('art .', 83778),
 ('ust .', 53552),
 ('poz .', 45198),
 (', poz', 43188),
 ('. 1', 39953),
 ('- -', 36547),
 ('r .', 33015),
 ('w art', 32042),
 (', o', 29926),
 ('mowa w', 28471),
 ('. 2', 26714),
 ('w ust', 23557),
 ('. art', 22922),
 (', w', 22495),
 ('. nr', 21433),
 ('2 .', 21285),
 ('1 .', 21267),
 (', z', 19680),
 (') w', 18311),
 ('. 3', 17167)]

3.Discard bigrams containing characters other than letters. Make sure that you discard the invalid entries after computing the bigram counts.

In [9]:
bigram_counts = {
    bigram: count
    for (bigram, count)
    in bigram_counts.items()
    if all([token.isalpha() for token in bigram.split(' ')])
}

In [10]:
sorted(bigram_counts.items(), key=lambda bigram_count: -bigram_count[1])[:20]


[('w art', 32042),
 ('mowa w', 28471),
 ('w ust', 23557),
 ('o których', 13884),
 ('których mowa', 13857),
 ('otrzymuje brzmienie', 9553),
 ('z dnia', 9527),
 ('o którym', 9184),
 ('którym mowa', 9171),
 ('do spraw', 8715),
 ('i nr', 8435),
 ('dodaje się', 8195),
 ('w brzmieniu', 7280),
 ('w drodze', 7127),
 ('na podstawie', 6674),
 ('stosuje się', 6535),
 ('w przypadku', 6021),
 ('o której', 5529),
 ('której mowa', 5510),
 ('w zakresie', 5454)]

4. Use pointwise mutual information to compute the measure for all pairs of words.

In [11]:
unigram_counts = collections.defaultdict(int)

for file, tokens in text_tokenized.items():
    for unigram, count in count_ngrams(tokens, n=1).items():
        unigram_counts[unigram] += count

unigram_counts = {
    unigram: count
    for (unigram, count)
    in unigram_counts.items()
    if unigram.isalpha()
}

In [12]:
sorted(unigram_counts.items(), key=lambda unigram_count: -unigram_count[1])[:20]


[('w', 201224),
 ('i', 90009),
 ('art', 83804),
 ('z', 82443),
 ('o', 64776),
 ('do', 60735),
 ('ust', 53636),
 ('na', 50647),
 ('się', 45886),
 ('lub', 45800),
 ('poz', 45224),
 ('nr', 44950),
 ('oraz', 33558),
 ('r', 33177),
 ('mowa', 28783),
 ('nie', 22990),
 ('przez', 20953),
 ('pkt', 19124),
 ('dnia', 17954),
 ('których', 17932)]

In [13]:
total_unigram_count = sum(unigram_counts.values())
total_bigram_count = sum(bigram_counts.values())

In [14]:
def pmi2(bigram, separator=' '):
    first_token, second_token = bigram.split(separator)
    first_token_p = unigram_counts[first_token] / total_unigram_count
    second_token_p = unigram_counts[second_token] / total_unigram_count
    bigram_p = bigram_counts[bigram] / total_bigram_count
    return math.log2(bigram_p / (first_token_p * second_token_p))

In [15]:
bigram_pmi = {
    bigram: pmi2(bigram)
    for (bigram, count)
    in bigram_counts.items()
}

In [16]:
list(bigram_pmi.items())[:10]


[('ustawa z', 4.375258347743362),
 ('z dnia', 4.903851080276138),
 ('o ratyfikacji', 5.17245512966816),
 ('ratyfikacji traktatu', 11.564000660854001),
 ('traktatu o', 4.118694392752504),
 ('o otwartych', 2.078537266280521),
 ('otwartych przestworzach', 17.061709028468528),
 ('przestworzach art', 4.794440384163163),
 ('upoważnia się', 6.333256839662903),
 ('się prezydenta', 1.3596246931781315)]

5. Sort the word pairs according to that measure in the descending order and determine top 10 entries.

In [17]:
sorted(bigram_pmi.items(), key=lambda bigram_pmi: -bigram_pmi[1])[:10]


[('ugaszone zapałki', 22.149171869718867),
 ('rozgarnia ściółkę', 22.149171869718867),
 ('reakcjami psychofizycznymi', 22.149171869718867),
 ('papaver somniferum', 22.149171869718867),
 ('makiem ogrodowym', 22.149171869718867),
 ('owocujące wierzchołki', 22.149171869718867),
 ('resztek pożniwnych', 22.149171869718867),
 ('dziele pielęgnowania', 22.149171869718867),
 ('virtus et', 22.149171869718867),
 ('et fraternitas', 22.149171869718867)]

6. Filter bigrams with number of occurrences lower than 5. Determine top 10 entries for the remaining dataset (>=5 occurrences).

In [18]:
filtered_bigram_pmi = {
    bigram: pmi
    for (bigram, pmi)
    in bigram_pmi.items()
    if bigram_counts[bigram] >= 5
}

In [19]:
sorted(filtered_bigram_pmi.items(), key=lambda bigram_pmi: -bigram_pmi[1])[:10]


[('świeckie przygotowujące', 19.827243774831505),
 ('klęskami żywiołowymi', 19.827243774831505),
 ('obcowania płciowego', 19.827243774831505),
 ('nietykalność cielesną', 19.827243774831505),
 ('młyny kulowe', 19.827243774831505),
 ('młynki młotkowe', 19.827243774831505),
 ('najnowszych zdobyczy', 19.827243774831505),
 ('teryto rialnego', 19.827243774831505),
 ('obiegów chłodzących', 19.827243774831505),
 ('otworami wiertniczymi', 19.827243774831505)]

7. Use KRNNT or Clarin-PL API(https://ws.clarin-pl.eu/tager.shtml) to tag and lemmatize the corpus.

In [20]:
!docker pull djstrong/krnnt2


Using default tag: latest
Got permission denied while trying to connect to the Docker daemon socket at unix:///var/run/docker.sock: Post "http://%2Fvar%2Frun%2Fdocker.sock/v1.24/images/create?fromImage=djstrong%2Fkrnnt2&tag=latest": dial unix /var/run/docker.sock: connect: permission denied


In [21]:
!docker run -it -p "9200:9200" djstrong/krnnt2

docker: Got permission denied while trying to connect to the Docker daemon socket at unix:///var/run/docker.sock: Post "http://%2Fvar%2Frun%2Fdocker.sock/v1.24/containers/create": dial unix /var/run/docker.sock: connect: permission denied.
See 'docker run --help'.


In [22]:
!python3 /home/krnnt/krnnt/krnnt_serve.py /home/krnnt/krnnt/data

python3: can't open file '/home/krnnt/krnnt/krnnt_serve.py': [Errno 2] No such file or directory


In [24]:
print(requests.post('http://localhost:9200', data='Ala ma kota.').text, end='')


Ala	none
	Ala	subst:sg:nom:f	disamb
ma	space
	mieć	fin:sg:ter:imperf	disamb
kota	space
	kot	subst:sg:acc:m2	disamb
.	none
	.	interp	disamb



In [26]:
print(list(lemmatized_acts.values())[0][:50])
lemmatized_acts_copy = lemmatized_acts


['dziennik:brev', '.:interp', 'ustawa:brev', '.:interp', 'z:prep', '1995:adj', 'rok:brev', '.:interp', 'numer:brev', '14:num', ',:interp', 'pozycja:brev', '.:interp', '62:adj', 'ustawa:subst', 'z:prep', 'dzień:subst', '15:adj', 'grudzień:subst', '1994:adj', 'rok:brev', '.:interp', 'o:prep', 'ratyfikacja:subst', 'traktat:subst', 'o:prep', 'otworzyć:ppas', 'przestworze:subst', 'artykuł:brev', '.:interp', '1:adj', '.:interp', 'upoważniać:fin', 'się:qub', 'prezydent:subst', 'rzeczpospolita:subst', 'polski:adj', 'do:prep', 'ratyfikować:ger', 'traktat:subst', 'o:prep', 'otwarty:adj', 'przestworze:subst', ',:interp', 'podpisać:ppas', 'w:prep', 'helsinki:subst', 'dzień:subst', '24:adj', 'marzec:subst']


8. Using the tagged corpus compute bigram statistic for the tokens containing:
 - a. lemmatized, downcased word
  - b. morphosyntactic category of the word (subst, fin, adj, etc.)

In [28]:
def count_ngrams(tokens, n=2):
    ngram_dict = collections.defaultdict(int)
    for words in zip(*[tokens[offset:] for offset in (range(n))]):
        ngram_dict[' '.join(words)] += 1
    return ngram_dict

In [29]:
lemmatized_bigram_counts = collections.defaultdict(int)

for _, lemmatized_act in lemmatized_acts.items():
    for bigram, count in count_ngrams(lemmatized_act).items():
        lemmatized_bigram_counts[bigram] += count

In [30]:
sorted(lemmatized_bigram_counts.items(), key=lambda bigram_count: -bigram_count[1])[:20]


[('artykuł:brev .:interp', 83738),
 ('ustęp:brev .:interp', 53315),
 ('pozycja:brev .:interp', 45197),
 (',:interp pozycja:brev', 43168),
 ('.:interp 1:adj', 39986),
 ('-:interp -:interp', 36548),
 ('rok:brev .:interp', 33021),
 ('w:prep artykuł:brev', 32035),
 (',:interp o:prep', 29913),
 ('o:prep który:adj', 28656),
 ('który:adj mowa:subst', 28538),
 ('mowa:subst w:prep', 28473),
 ('.:interp 2:adj', 26408),
 ('w:prep ustęp:brev', 23536),
 ('.:interp artykuł:brev', 22917),
 (',:interp w:prep', 22544),
 ('.:interp numer:brev', 21433),
 ('1:adj .:interp', 21311),
 ('2:adj .:interp', 21119),
 (',:interp z:prep', 20015)]

9. Discard bigrams containing characters other than letters

For example: "Ala ma kota", which is tagged as:
   ```
   Ala	none
           Ala	subst:sg:nom:f	disamb
   ma	space
           mieć	fin:sg:ter:imperf	disamb
   kota	space
           kot	subst:sg:acc:m2	disamb
   .	none
           .	interp	disamb
   ```
   the algorithm should return the following bigrams: `ala:subst mieć:fin` and `mieć:fin kot:subst`.

In [31]:
lemmatized_bigram_counts = {
    bigram: count
    for (bigram, count)
    in lemmatized_bigram_counts.items()
    if all([token.split(':')[0].isalpha() and token != 'X:X' for token in bigram.split(' ')])
}

In [32]:
sorted(lemmatized_bigram_counts.items(), key=lambda bigram_count: -bigram_count[1])[:20]


[('w:prep artykuł:brev', 32035),
 ('o:prep który:adj', 28656),
 ('który:adj mowa:subst', 28538),
 ('mowa:subst w:prep', 28473),
 ('w:prep ustęp:brev', 23536),
 ('z:prep dzień:subst', 11360),
 ('otrzymywać:fin brzmienie:subst', 10535),
 ('określić:ppas w:prep', 9715),
 ('do:prep sprawa:subst', 8718),
 ('ustawa:subst z:prep', 8625),
 ('właściwy:adj do:prep', 8535),
 ('i:conj numer:brev', 8435),
 ('dodawać:fin się:qub', 8196),
 ('minister:subst właściwy:adj', 7933),
 ('w:prep brzmienie:subst', 7277),
 ('w:prep droga:subst', 7128),
 ('w:prep przypadek:subst', 6776),
 ('na:prep podstawa:subst', 6681),
 ('stosować:fin się:qub', 6535),
 ('się:qub wyraz:subst', 6077)]

10. Compute the same statistics as for the non-lemmatized words (i.e. PMI) and print top-10 entries with at least 5 occurrences.

In [56]:
lemmatized_unigram_counts = collections.defaultdict(int)

for file, tokens in lemmatized_acts.items():
    for unigram, count in count_ngrams(tokens, n=1).items():
        lemmatized_unigram_counts[unigram] += count

lemmatized_unigram_counts = {
    unigram: count
    for (unigram, count)
    in lemmatized_unigram_counts.items()
    if all([token.split(':')[0].isalpha() and token != 'X:X' for token in unigram.split(' ')])

}

In [57]:
sorted(lemmatized_unigram_counts.items(), key=lambda unigram_count: -unigram_count[1])[:20]


[('w:prep', 202615),
 ('i:conj', 89442),
 ('z:prep', 87987),
 ('artykuł:brev', 83742),
 ('o:prep', 64713),
 ('do:prep', 60761),
 ('ustęp:brev', 53335),
 ('na:prep', 50651),
 ('który:adj', 49380),
 ('się:qub', 45888),
 ('lub:conj', 45800),
 ('pozycja:brev', 45197),
 ('numer:brev', 44868),
 ('oraz:conj', 33564),
 ('rok:brev', 33050),
 ('mowa:subst', 28804),
 ('dzień:subst', 27885),
 ('on:ppron3', 26330),
 ('ustawa:subst', 24933),
 ('nie:qub', 22748)]

In [58]:
total_lemmitized_unigram_count = sum(lemmatized_unigram_counts.values())
total_lemmitized_bigram_count = sum(lemmatized_bigram_counts.values())

In [59]:
def lemmitized_pmi2(bigram, separator=' '):
    first_token, second_token = bigram.split(separator)
    first_token_p = lemmatized_unigram_counts[first_token] / total_lemmitized_unigram_count
    second_token_p = lemmatized_unigram_counts[second_token] / total_lemmitized_unigram_count
    bigram_p = lemmatized_bigram_counts[bigram] / total_lemmitized_bigram_count
    return math.log2(bigram_p / (first_token_p * second_token_p))

In [60]:
lemmatized_bigram_pmi = {
    bigram: lemmitized_pmi2(bigram)
    for (bigram, count)
    in lemmatized_bigram_counts.items()
}


In [63]:
lemmatized_bigram_pmi

{'ustawa:subst z:prep': 4.194234150253142,
 'z:prep dzień:subst': 4.430167834122352,
 'o:prep ratyfikacja:subst': 5.162559567983826,
 'ratyfikacja:subst traktat:subst': 11.304248987681536,
 'traktat:subst o:prep': 3.885995129588055,
 'o:prep otworzyć:ppas': -0.09785744783534765,
 'otworzyć:ppas przestworze:subst': 14.883910491715254,
 'przestworze:subst artykuł:brev': 4.797033278660855,
 'upoważniać:fin się:qub': 6.2601721686508665,
 'się:qub prezydent:subst': 0.5930478602104122,
 'prezydent:subst rzeczpospolita:subst': 9.080420348845651,
 'rzeczpospolita:subst polski:adj': 9.054383699450149,
 'polski:adj do:prep': -0.7758221452800045,
 'do:prep ratyfikować:ger': 6.107835945613037,
 'ratyfikować:ger traktat:subst': 12.638944378642776,
 'o:prep otwarty:adj': -0.0009959085827589633,
 'otwarty:adj przestworze:subst': 13.395809530246687,
 'podpisać:ppas w:prep': 3.6984725293901377,
 'w:prep helsinki:subst': 4.522315574236311,
 'helsinki:subst dzień:subst': 7.383495381882156,
 'ustawa:subst

11. Compute trigram counts for both corpora and perform the same filtering.

### Trigrams

In [64]:
trigram_counts = collections.defaultdict(int)

for file, tokens in text_tokenized.items():
    for trigram, count in count_ngrams(tokens, n=3).items():
        trigram_counts[trigram] += count

In [65]:
sorted(trigram_counts.items(), key=lambda trigram_count: -trigram_count[1])[:20]


[(', poz .', 43166),
 ('- - -', 34646),
 ('w art .', 32032),
 ('w ust .', 23520),
 ('ust . 1', 23324),
 ('. art .', 22917),
 ('r . nr', 17860),
 ('_ _ _', 16213),
 ('. 1 .', 15635),
 ('. 2 .', 15282),
 ('o których mowa', 13856),
 ('których mowa w', 13806),
 (', o których', 13774),
 ('mowa w ust', 13474),
 (': 1 )', 13465),
 ('mowa w art', 12311),
 (') w art', 11719),
 ('ust . 2', 10746),
 ('. 3 .', 9673),
 ('otrzymuje brzmienie :', 9505)]

In [66]:
trigram_counts = {
    trigram: count
    for (trigram, count)
    in trigram_counts.items()
    if all([token.isalpha() for token in trigram.split(' ')])
}

In [67]:
sorted(trigram_counts.items(), key=lambda trigram_count: -trigram_count[1])[:20]


[('o których mowa', 13856),
 ('których mowa w', 13806),
 ('mowa w ust', 13474),
 ('mowa w art', 12311),
 ('o którym mowa', 9169),
 ('którym mowa w', 9147),
 ('o której mowa', 5510),
 ('której mowa w', 5487),
 ('w drodze rozporządzenia', 4685),
 ('właściwy do spraw', 4620),
 ('minister właściwy do', 4600),
 ('ustawie z dnia', 3649),
 ('w ustawie z', 3645),
 ('stosuje się odpowiednio', 3089),
 ('ustawy z dnia', 3053),
 ('zastępuje się wyrazami', 2940),
 ('dodaje się ust', 2766),
 ('wejścia w życie', 2353),
 ('dni od dnia', 2070),
 ('się następujące zmiany', 1806)]

### Lemmatized Trigrams

In [80]:
lemmatized_trigram_counts = collections.defaultdict(int)

for file, tokens in lemmatized_acts.items():
    for trigram, count in count_ngrams(tokens, n=3).items():
        lemmatized_trigram_counts[trigram] += count

In [81]:
sorted(lemmatized_trigram_counts.items(), key=lambda trigram_count: -trigram_count[1])[:20]


[(',:interp pozycja:brev .:interp', 43168),
 ('-:interp -:interp -:interp', 34646),
 ('w:prep artykuł:brev .:interp', 32031),
 ('o:prep który:adj mowa:subst', 28535),
 (',:interp o:prep który:adj', 28455),
 ('który:adj mowa:subst w:prep', 28442),
 ('w:prep ustęp:brev .:interp', 23520),
 ('ustęp:brev .:interp 1:adj', 23310),
 ('.:interp artykuł:brev .:interp', 22917),
 ('rok:brev .:interp numer:brev', 17860),
 ('.:interp .:interp .:interp', 17153),
 ('_:interp _:interp _:interp', 16213),
 ('.:interp 1:adj .:interp', 15643),
 ('.:interp 2:adj .:interp', 15280),
 ('::interp 1:adj ):interp', 13489),
 ('mowa:subst w:prep ustęp:brev', 13462),
 ('brzmienie:subst ::interp ":interp', 12499),
 ('mowa:subst w:prep artykuł:brev', 12307),
 ('):interp w:prep artykuł:brev', 12052),
 ('ustęp:brev .:interp 2:adj', 10502)]

In [82]:
lemmatized_trigram_counts = {
    trigram: count
    for (trigram, count)
    in lemmatized_trigram_counts.items()
    if all([token.split(':')[0].isalpha() and token != 'X:X' for token in trigram.split(' ')])
}

In [83]:
sorted(lemmatized_trigram_counts.items(), key=lambda trigram_count: -trigram_count[1])[:20]


[('o:prep który:adj mowa:subst', 28535),
 ('który:adj mowa:subst w:prep', 28442),
 ('mowa:subst w:prep ustęp:brev', 13462),
 ('mowa:subst w:prep artykuł:brev', 12307),
 ('ustawa:subst z:prep dzień:subst', 8589),
 ('właściwy:adj do:prep sprawa:subst', 7966),
 ('minister:subst właściwy:adj do:prep', 7888),
 ('w:prep droga:subst rozporządzenie:subst', 4748),
 ('zastępować:fin się:qub wyraz:subst', 3653),
 ('w:prep ustawa:subst z:prep', 3646),
 ('stosować:fin się:qub odpowiednio:adv', 3089),
 ('określić:ppas w:prep artykuł:brev', 3071),
 ('dodawać:fin się:qub ustęp:brev', 2699),
 ('wejść:ger w:prep życie:subst', 2381),
 ('dzień:subst wejść:ger w:prep', 2099),
 ('dzień:subst od:prep dzień:subst', 2093),
 ('wchodzić:fin w:prep życie:subst', 1701),
 ('w:prep porozumienie:subst z:prep', 1667),
 ('określić:ppas w:prep ustęp:brev', 1622),
 ('wprowadzać:fin się:qub następujący:adj', 1613)]

12. Use PMI (with 5 occurrence threshold) to compute top 10 results for the trigrams. Devise a method for computing the values, based on the results for bigrams.

In [88]:
total_trigram_count = sum(trigram_counts.values())
total_lemmitized_trigram_count = sum(lemmatized_trigram_counts.values())

# log2(P(x, y, z) / (P(x) * P(y) * P(z)))
def pmi3(trigram, separator=' '):
    first_token, second_token, third_token = trigram.split(separator)
    first_token_p = unigram_counts[first_token] / total_unigram_count
    second_token_p = unigram_counts[second_token] / total_unigram_count
    third_token_p = unigram_counts[third_token] / total_unigram_count
    trigram_p = trigram_counts[trigram] / total_trigram_count
    return math.log2(trigram_p / (first_token_p * second_token_p * third_token_p))

# log2(P(x, y, z) / (P(x) * P(y) * P(z)))
def lemmatized_pmi3(trigram, separator=' '):
    first_token, second_token, third_token = trigram.split(separator)
    first_token_p = lemmatized_unigram_counts[first_token] / total_lemmitized_unigram_count
    second_token_p = lemmatized_unigram_counts[second_token] / total_lemmitized_unigram_count
    third_token_p = lemmatized_unigram_counts[third_token] / total_lemmitized_unigram_count
    trigram_p = lemmatized_trigram_counts[trigram] / total_lemmitized_trigram_count
    return math.log2(trigram_p / (first_token_p * second_token_p * third_token_p))

### Trigrams

In [85]:
trigram_pmi = {
    trigram: pmi3(trigram)
    for (trigram, count)
    in trigram_counts.items()
}

sorted(trigram_pmi.items(), key=lambda trigram_pmi: -trigram_pmi[1])[:10]

[('virtus et fraternitas', 44.21571342809874),
 ('salus aegroti suprema', 44.21571342809874),
 ('aegroti suprema lex', 44.21571342809874),
 ('ośmioramienna gwiazda policyjna', 44.21571342809874),
 ('przekazowi pieniężnemu towarzyszyły', 44.21571342809874),
 ('prawosławnym metropolitą warszawskim', 44.21571342809874),
 ('mitteldeutsche grundstucksgesellschaft mit', 44.21571342809874),
 ('grundstucksgesellschaft mit beschrankter', 44.21571342809874),
 ('mit beschrankter haftung', 44.21571342809874),
 ('fabryki portland cementu', 44.21571342809874)]

In [86]:
filtered_trigram_pmi = {
    trigram: pmi
    for (trigram, pmi)
    in trigram_pmi.items()
    if trigram_counts[trigram] >= 5
}

sorted(filtered_trigram_pmi.items(), key=lambda trigram_pmi: -trigram_pmi[1])[:10]

[('finałowego turnieju mistrzostw', 37.05584209132035),
 ('profilem zaufanym epuap', 36.814833991816556),
 ('cienką sierścią zwierzęcą', 36.75628180946144),
 ('przedwczesnego wyrębu drzewostanu', 36.63075092737758),
 ('centralnemu biuru antykorupcyjnemu', 36.36396438668268),
 ('turnieju mistrzostw europy', 36.29030734495737),
 ('potwierdzonym profilem zaufanym', 36.26734619651406),
 ('szybkiemu postępowi technicznemu', 36.138897831047906),
 ('piłce nożnej uefa', 36.119789008100206),
 ('wypalonym paliwem jądrowym', 35.91650540971146)]

### Lemmatized Trigrams

In [89]:
lemmatized_trigram_pmi = {
    trigram: lemmatized_pmi3(trigram)
    for (trigram, count)
    in lemmatized_trigram_counts.items()
}

sorted(lemmatized_trigram_pmi.items(), key=lambda trigram_pmi: -trigram_pmi[1])[:10]

[('salus:subst aegroti:xxx suprema:subst', 44.21827330917656),
 ('aegroti:xxx suprema:subst lex:xxx', 44.21827330917656),
 ('mitteldeutsche:subst grundstucksgesellschaft:subst mit:subst',
  44.21827330917656),
 ('grundstucksgesellschaft:subst mit:subst beschrankter:subst',
  44.21827330917656),
 ('mit:subst beschrankter:subst haftung:subst', 44.21827330917656),
 ('asistent:subst medical:subst obstetrică:adja', 44.21827330917656),
 ('staphylococcus:subst aureus:subst wankomycynooporne:xxx',
  44.21827330917656),
 ('dosuszać:ger blaszka:subst liściowy:adj', 44.21827330917656),
 ('chleb:subst świętojański:adj strąkowy:adj', 44.21827330917656),
 ('wszywać:ger zamek:subst błyskawiczny:adj', 44.21827330917656)]

In [90]:
filtered_lemmatized_trigram_pmi = {
    trigram: pmi
    for (trigram, pmi)
    in lemmatized_trigram_pmi.items()
    if lemmatized_trigram_counts[trigram] >= 5
}

sorted(filtered_lemmatized_trigram_pmi.items(), key=lambda trigram_pmi: -trigram_pmi[1])[:10]

[('porcelanowy:adj młyn:subst kulowy:adj', 38.08899029223159),
 ('wymiennik:subst przeponowy:adj rurowy:adj', 37.21827330917656),
 ('finałowy:adj turniej:subst mistrzostwa:subst', 36.2239198723177),
 ('vitis:subst vinifera:subst l:brev', 36.05840197239817),
 ('piłka:subst nożny:adj uefa:subst', 35.4593838764556),
 ('turniej:subst mistrzostwa:subst europa:subst', 35.022286011148054),
 ('przedwczesny:adj wyrąb:subst drzewostan:subst', 35.00395418837579),
 ('mecz:subst piłka:subst nożny:adj', 34.72241828228939),
 ('profil:subst zaufany:adj epuap:subst', 34.63519054167363),
 ('milion:brev dolar:subst usa:subst', 34.47343947167702)]

14. Answer the following questions:
  a) Why do we have to filter the bigrams, rather than the token sequence?

So that we don't modify the source text and only considering the result we get from it.

  b) Which method works better for the bigrams and which for the trigrams?

PMI looks to work well for both cases

  c) What types of expressions are discovered by the methods.

PMI finds the sets in which the meaning is matched very well. It gives the highest score to words that have particularly stronged meaning togethe.

  d) Can you devise a different type of filtering that would yield better results?

For example filtering out all prepositions or words that have no real meaning in the context.